In [6]:
# Librerías principales
import pandas as pd
import numpy as np
import os


In [3]:
# Definir la ruta al archivo
ruta_dta = "C:/SEM 8/Bloque dos & materias final/Proyecto BeWay/tablas & codigos/dtas/BASE_ENSAFI.dta"

# Leer el archivo .dta
data = pd.read_stata(ruta_dta)

# Ver las primeras filas para asegurarnos
#print(data.head())


In [7]:
# Definir ruta de salida
output_path = os.path.join('..', 'outputspy', 'vars_list.txt')

# Extraer los nombres de las variables
variable_names = data.columns.tolist()

# Escribir (sobrescribir) el archivo txt
with open(output_path, 'w', encoding='utf-8') as f:
    for var in variable_names:
        f.write(f"{var}\n")

print(f"Archivo 'vars_list.txt' creado exitosamente con {len(variable_names)} variables.")


Archivo 'vars_list.txt' creado exitosamente con 336 variables.
